# Data Sourcing

In [2]:
import pkg_resources
import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
required = {'researchpy', 'missingno', 'folium', 'pydotplus','bokeh','imblearn', 'catboost'}
missing = required - installedPackages
if missing:
    !pip install researchpy
    !pip install missingno
    !pip install folium
    !pip install pydotplus
    !pip install bokeh
    !pip install imblearn
    !pip install catboost
    #!pip install xgboost
    #!pip install graphviz

In [3]:
#Disable the warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
import timeit
start_time = timeit.default_timer()  #timestamp to calculate total runtime

import pandas as pd
import numpy as np

import researchpy as rp
import missingno as msno
import itertools
import scipy.stats as ss

import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium import plugins
import graphviz

from sklearn import tree
from sklearn import feature_selection
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, accuracy_score, \
precision_score, recall_score, roc_auc_score, f1_score, precision_recall_curve, auc 

%matplotlib inline
pd.set_option('display.max_columns', None)  # prevent column output trancation
sns.set()  # change plot styling from Matlab's 90s feel to today's Seaborn.

In [5]:
# File Directories
path_crashes = '../crashes.sample2020.csv'
path_vehicles = '../vehicles.sample.csv'
path_people = '../people.sample.csv'

# Import samples
crashes = pd.read_csv(path_crashes, parse_dates=["CRASH_DATE", "CRASH_DATE_EST_I", "DATE_POLICE_NOTIFIED"],
                      low_memory=False, dtype=object)
vehicles = pd.read_csv(path_vehicles, parse_dates=["CRASH_DATE"], low_memory=False, dtype=object)
people = pd.read_csv(path_people, parse_dates=["CRASH_DATE"], low_memory=False, dtype=object)

In [6]:
# Joining datasets
non_passengers=people[people.PERSON_ID.str.contains('^O')]

vehicles_with_people=vehicles.merge(non_passengers,how='left',on=['CRASH_RECORD_ID','RD_NO','CRASH_DATE','VEHICLE_ID'])

data=crashes.merge(vehicles_with_people,how='inner',on=['CRASH_RECORD_ID','RD_NO','CRASH_DATE'])

# Feature Selection
filter_list=["AGE","LANE_CNT","AIRBAG_DEPLOYED","PRIM_CONTRIBUTORY_CAUSE","POSTED_SPEED_LIMIT","NUM_UNITS","TRAFFICWAY_TYPE",  
             "SEC_CONTRIBUTORY_CAUSE","FIRST_CRASH_TYPE","MOST_SEVERE_INJURY","LIGHTING_CONDITION","SEX","CRASH_DATE",
             "CRASH_HOUR","VEHICLE_YEAR"]

# Data that will be used in predictions
modeling_data=data[filter_list]

In [7]:
# Split modeling data into raw train and test sets
raw_train, raw_test = train_test_split(modeling_data, test_size=0.20, random_state=42, shuffle=True)

# Data Preprocessing

## Preprocessing Train Set

In [8]:
def preprocessor(dataframe):
    '''Preprocesses df and returns X and y ready for modeling (after imputation of numericals!)'''
    df = dataframe.copy()
    
    # Prepare data for missing value imputation
    df.loc[df["LIGHTING_CONDITION"] == "UNKNOWN", "LIGHTING_CONDITION"] = np.nan
    df.loc[df["TRAFFICWAY_TYPE"] == "UNKNOWN","TRAFFICWAY_TYPE"] = np.nan
    df.loc[df["AIRBAG_DEPLOYED"] == "DEPLOYMENT UNKNOWN","AIRBAG_DEPLOYED"] = np.nan
    df.fillna({'LIGHTING_CONDITION': 'DAYLIGHT', 'TRAFFICWAY_TYPE': 'NOT DIVIDED',
               'SEX': 'UNABLE TO DETERMINE', 'AIRBAG_DEPLOYED': 'UNABLE TO DETERMINE'}, inplace=True)
    
    # Remove rows missing most severe injury results
    drop_rows = ['MOST_SEVERE_INJURY']
    df.dropna(how ='any', subset = drop_rows, inplace = True)
    
    # Handle numerical features
    df['VEHICLE_YEAR'] = pd.to_numeric(df['VEHICLE_YEAR'])
    df['NUM_UNITS'] = pd.to_numeric(df['NUM_UNITS'])
    df["POSTED_SPEED_LIMIT"] = pd.to_numeric(df["POSTED_SPEED_LIMIT"])
    df["AGE"] = pd.to_numeric(df["AGE"])
    
    df['LANE_CNT'] = pd.to_numeric(df['LANE_CNT'])    
    df['LANE_CNT'].fillna(2, inplace=True)
    df.loc[(df['LANE_CNT'] > 6),'LANE_CNT'] = 6
    
    # Function definitions
    def injury(x): 
        if any(s in x for s in ["FATAL","NONINCAPACITATING INJURY","INCAPACITATING INJURY"]):
            return "INJURED"
        else:
            return "NOT INJURED"
    
    def airbag(x):
        if ("DEPLOY" in x) and ("UNKNOWN" not in x):
            if "NOT" in x:
                return "NOT DEPLOYED"
            else:
                return "DEPLOYED"
        else:
            return x

    def crash_hour(x):
        if  2 <= x < 8:
            return "Early_morning"
        elif 8 <= x < 12:
            return "Morning"
        elif 12 <= x < 18:
            return "Afternoon"
        else:
            return "Night"
  
    def traffic_way(x):
        if ("NOT" in x) or ("ONE-WAY" in x):
            return "NOT_DIVIDED"
        else:
            return "DIVIDED"
    
    # Feature Engineering
    df["INJURY"] = df["MOST_SEVERE_INJURY"].apply(lambda x: injury(x))
    df["AIRBAG_DEPLOYED"] = df["AIRBAG_DEPLOYED"].apply(lambda x: airbag(x))
    df["CRASH_HOUR"] = df["CRASH_HOUR"].apply(lambda x: crash_hour(int(x)))
    df["TRAFFICWAY_TYPE"] = df["TRAFFICWAY_TYPE"].apply(lambda x: traffic_way(x))
    
    df["VEHICLE_AGE"] = df["CRASH_DATE"].dt.year-df["VEHICLE_YEAR"]
    df.loc[df["VEHICLE_AGE"] < 0, "VEHICLE_AGE"] = 0
    df.drop(["VEHICLE_YEAR", "CRASH_DATE", "MOST_SEVERE_INJURY"], axis=1, inplace=True)
    
    # Splitting df into X and y
    y = df["INJURY"]
    X = df.drop(["INJURY"], axis=1)
    
    # Binarize y
    from sklearn.preprocessing import label_binarize
    y = preprocessing.label_binarize(y, classes=['NOT INJURED', 'INJURED'])
    
    # One-Hot Encoding
    X = pd.get_dummies(X, columns = X.select_dtypes(['object']).columns)
    dummies_to_drop = X.columns[X.columns.str.contains("UNABLE|UNKNOWN|NOT APPLICABLE|OTHER")]
    X = X.loc[:, ~X.columns.isin(dummies_to_drop)]
      
    return(X, y)

In [9]:
# Preprocess train set
X_train, y_train = preprocessor(raw_train)

In [10]:
# Fit imputer and impute missing values on train set 
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)
df = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)
X_train = df.copy()
X_train = np.round(X_train)

In [11]:
X_train.isna().any().sum()

0

## Preprocessing Test Set

In [12]:
# Preprocess test set
X_test, y_test = preprocessor(raw_test)

In [13]:
# Add two trivial features of train set that doesn't exist in test set in order to use pre-fit imputer
X_test["PRIM_CONTRIBUTORY_CAUSE_OBSTRUCTED CROSSWALKS"] = 0
X_test["SEC_CONTRIBUTORY_CAUSE_PASSING STOPPED SCHOOL BUS"] = 0
X_test = X_test[X_train.columns]

In [14]:
# Impute missing values on test set by pre-fit imputer object 
df = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)
X_test = df.copy()
X_test = np.round(X_test)

In [15]:
X_test.isna().any().sum()

0

## Preprocessing EDA Train Set

In [16]:
def eda_preprocessor(dataframe):
    '''Preprocesses and returns the entire train set for EDA'''
    
    df = dataframe.copy()
    
    # Split data into train test with exact indexing
    train, test = train_test_split(df, test_size=0.20, random_state=42, shuffle=True)
    
    # Feature reduction
    drop=["CRASH_DATE_EST_I", "REPORT_TYPE", "CRASH_DATE_EST_I", "REPORT_TYPE", "DATE_POLICE_NOTIFIED",
          "BEAT_OF_OCCURRENCE", "PHOTOS_TAKEN_I", "STATEMENTS_TAKEN_I", "WORK_ZONE_TYPE", "WORKERS_PRESENT_I", 
          "INJURIES_NO_INDICATION", "INJURIES_UNKNOWN", "CRASH_DAY_OF_WEEK", "CRASH_MONTH", "RD_NO", "VEHICLE_ID", 
          "CRASH_RECORD_ID", "SEAT_NO", "STATE", "ZIPCODE", "DRIVERS_LICENSE_STATE", "PERSON_ID", "DRIVERS_LICENSE_CLASS", 
          "INJURY_CLASSIFICATION", "HOSPITAL", "EMS_AGENCY", "EMS_RUN_NO", "PEDPEDAL_LOCATION", "LOCATION",
          "DAMAGE", "CRASH_TYPE", "MODEL", "BAC_RESULT", "CRASH_UNIT_ID", "RD_NO", "UNIT_NO", "UNIT_TYPE", 
          "VEHICLE_ID", "CMRC_VEH_I", "MAKE", "TOWED_I", "FIRE_I", "TOWED_BY", "STREET_NO","STREET_NAME",
          "TOWED_TO", "AREA_00_I", "AREA_01_I", "AREA_02_I", "AREA_03_I", "AREA_04_I", "AREA_05_I", "AREA_06_I", 
          "AREA_07_I", "AREA_08_I", "AREA_09_I", "AREA_10_I", "AREA_11_I", "AREA_12_I", "AREA_99_I", "CMV_ID", 
          "USDOT_NO", "CCMC_NO", "ILCC_NO", "COMMERCIAL_SRC", "GVWR", "CARRIER_NAME", "CARRIER_STATE", 
          "CARRIER_CITY", "HAZMAT_PLACARDS_I", "HAZMAT_NAME", "UN_NO", "HAZMAT_PRESENT_I", "HAZMAT_REPORT_I",
          "HAZMAT_REPORT_NO", "MCS_REPORT_I", "MCS_REPORT_NO", "HAZMAT_VIO_CAUSE_CRASH_I", "MCS_VIO_CAUSE_CRASH_I", 
          "IDOT_PERMIT_NO", "WIDE_LOAD_I", "TRAILER1_WIDTH", "TRAILER2_WIDTH", "TRAILER1_LENGTH", "TRAILER2_LENGTH", 
          "TOTAL_VEHICLE_LENGTH", "AXLE_CNT", "VEHICLE_CONFIG", "CARGO_BODY_TYPE", "LOAD_TYPE", "HAZMAT_OUT_OF_SERVICE_I",
          "INJURIES_REPORTED_NOT_EVIDENT", "INJURIES_NON_INCAPACITATING", #"INJURIES_FATAL","INJURIES_INCAPACITATING",
          "MCS_OUT_OF_SERVICE_I", "HAZMAT_CLASS"]
    
    train = train.loc[:, ~train.columns.isin(drop)]
    
    
    # Prepare data for missing value imputation
    train.loc[train["TRAFFIC_CONTROL_DEVICE"]=="UNKNOWN","TRAFFIC_CONTROL_DEVICE"]=np.nan
    train.loc[train["DEVICE_CONDITION"]=="UNKNOWN","DEVICE_CONDITION"]=np.nan
    train.loc[train["WEATHER_CONDITION"]=="UNKNOWN","WEATHER_CONDITION"]=np.nan
    train.loc[train["LIGHTING_CONDITION"]=="UNKNOWN","LIGHTING_CONDITION"]=np.nan
    train.loc[train["TRAFFICWAY_TYPE"]=="UNKNOWN","TRAFFICWAY_TYPE"]=np.nan
    train.loc[train["ROADWAY_SURFACE_COND"]=="UNKNOWN","ROADWAY_SURFACE_COND"]=np.nan
    train.loc[train["ROAD_DEFECT"]=="UNKNOWN","ROAD_DEFECT"]=np.nan

    train.loc[train["VEHICLE_DEFECT"]=="UNKNOWN","VEHICLE_DEFECT"]=np.nan
    train.loc[train["VEHICLE_TYPE"]=="UNKNOWN/NA","VEHICLE_TYPE"]=np.nan
    train.loc[train["TRAVEL_DIRECTION"]=="UNKNOWN","TRAVEL_DIRECTION"]=np.nan
    train.loc[train["MANEUVER"]=="UNKNOWN/NA","MANEUVER"]=np.nan

    train.loc[train["SAFETY_EQUIPMENT"]=="USAGE UNKNOWN","SAFETY_EQUIPMENT"]=np.nan
    train.loc[train["AIRBAG_DEPLOYED"]=="DEPLOYMENT UNKNOWN","AIRBAG_DEPLOYED"]=np.nan
    train.loc[train["EJECTION"]=="UNKNOWN","EJECTION"]=np.nan
    train.loc[train["DRIVER_ACTION"]=="UNKNOWN","DRIVER_ACTION"]=np.nan
    train.loc[train["DRIVER_VISION"]=="UNKNOWN","DRIVER_VISION"]=np.nan
    train.loc[train["PHYSICAL_CONDITION"]=="UNKNOWN","PHYSICAL_CONDITION"]=np.nan
    train.loc[train["PEDPEDAL_ACTION"]=="UNKNOWN/NA","PEDPEDAL_ACTION"]=np.nan
    
    
    # Mıssing Value Imputation
    train.fillna({
        'TRAFFIC_CONTROL_DEVICE': 'NO CONTROLS',
        'DEVICE_CONDITION': 'NO CONTROLS',
        'WEATHER_CONDITION': 'CLEAR',
        'LIGHTING_CONDITION': 'DAYLIGHT', # ??
        'TRAFFICWAY_TYPE': 'NOT DIVIDED', # ??
        'ROADWAY_SURFACE_COND': 'NO DEFECTS',
        'ROAD_DEFECT': 'CLEAR',
        'INTERSECTION_RELATED_I': 'N',
    
        'NOT_RIGHT_OF_WAY_I': 'N',
        'HIT_AND_RUN_I': 'N',
        'DOORING_I': 'N',
        'WORK_ZONE_I': 'N',
        'NUM_PASSENGERS': 0,
        'LIC_PLATE_STATE': 'IL',
        'VEHICLE_DEFECT': 'UNABLE TO DETERMINE',
        'VEHICLE_TYPE': 'OTHER',

        'VEHICLE_USE': 'OTHER',
        'TRAVEL_DIRECTION': 'N',
        'MANEUVER': 'OTHER',
        'OCCUPANT_CNT': 0,
        'EXCEED_SPEED_LIMIT_I': 'N',
        'FIRST_CONTACT_POINT': 'OTHER',
        'PERSON_TYPE': 'UNABLE TO DETERMINE',
        'CITY': 'OTHER',

        'SEX': 'UNABLE TO DETERMINE',
        'AIRBAG_DEPLOYED': 'UNABLE TO DETERMINE',
        'EJECTION': 'UNABLE TO DETERMINE',
        'DRIVER_ACTION': 'OTHER',
        'DRIVER_VISION': 'OTHER',
        'PHYSICAL_CONDITION': 'UNABLE TO DETERMINE',
        'PEDPEDAL_ACTION': 'UNABLE TO DETERMINE',
        'PEDPEDAL_VISIBILITY': 'UNABLE TO DETERMINE',

        'CELL_PHONE_USE': 'UNABLE TO DETERMINE',
        'SAFETY_EQUIPMENT': 'UNABLE TO DETERMINE',
        'BAC_RESULT VALUE': 0
    }, inplace=True)
    
    
    # Drop rows with missing values in these features
    drop_rows = ['INJURIES_TOTAL', 'LATITUDE', 'MOST_SEVERE_INJURY']
    train.dropna(how = 'any', subset = drop_rows, inplace = True)
    

    # Handle numerical features
    train['LANE_CNT'] = pd.to_numeric(train['LANE_CNT'])
    train['VEHICLE_YEAR'] = pd.to_numeric(train['VEHICLE_YEAR'])
    train['NUM_UNITS'] = pd.to_numeric(train['NUM_UNITS'])
    train["POSTED_SPEED_LIMIT"] = pd.to_numeric(train["POSTED_SPEED_LIMIT"])
    train["AGE"] = pd.to_numeric(train["AGE"])
    train["INJURIES_FATAL"] = pd.to_numeric(train["INJURIES_FATAL"])
    train["INJURIES_INCAPACITATING"] = pd.to_numeric(train["INJURIES_INCAPACITATING"])
    train["INJURIES_TOTAL"] = pd.to_numeric(train["INJURIES_TOTAL"])
    
    train['TOTAL_FATAL'] = train['INJURIES_FATAL'] + train['INJURIES_INCAPACITATING']
    train['TOTAL_FATAL'] = pd.to_numeric(train['TOTAL_FATAL'])

    train['LANE_CNT'] = pd.to_numeric(train['LANE_CNT'])    
    train['LANE_CNT'].fillna(2, inplace=True)
    train.loc[(train['LANE_CNT'] > 6),'LANE_CNT'] = 6
    
    
    # Function definitions
    def injury(x): 
        if any(s in x for s in ["FATAL","NONINCAPACITATING INJURY","INCAPACITATING INJURY"]):
            return "INJURED"
        else:
            return "NOT INJURED"
        
    def fatalities(x): 
        if any(s in x for s in ["FATAL"]):
            return "FATAL"
        else:
            return "NOT FATAL"
    
    def airbag(x):
        if ("DEPLOY" in x) and ("UNKNOWN" not in x):
            if "NOT" in x:
                return "NOT DEPLOYED"
            else:
                return "DEPLOYED"
        else:
            return x
  
    def traffic_way(x):
        if ("NOT" in x) or ("ONE-WAY" in x):
            return "NOT_DIVIDED"
        else:
            return "DIVIDED"
        
    def contact_point(x):
        if "FRONT" in x:
            return "FRONT"
        elif "SIDE" in x:
            return "SIDE"
        elif "REAR" in x:
            return "REAR"
        else:
            return "OTHER"
    
    def equip_used(x):
        if ("USED" in x) or ("HELMET" in x) or ("NONE PRESENT" in x):
            if any(s in x for s in ["NOT","IMPROPER","NONE PRESENT"]):
                return "DID NOT USE SAFETY EQUIP"
            else:
                return "USED SAFETY EQUIP"
        else:
            return x

    def airbag(x):
        if ("DEPLOY" in x) and ("UNKNOWN" not in x):
            if "NOT" in x:
                return "NOT DEPLOYED"
            else:
                return "DEPLOYED"
        else:
            return x
    
    def crash_hour(x):
        if  2 <= x < 8:
            return "Early_morning"
        elif 8 <= x < 12:
            return "Morning"
        elif 12 <= x < 18:
            return "Afternoon"
        else:
            return "Night"

    def traffic_control(x):
        if ("NO CONTROLS" in x) or ("UNKNOWN" in x) or ("OTHER" in x):
            return "NO_SIGN"
        else:
            return "SIGN"
    
    def location(x1,x2):
        if (41.84 <= float(x1) <= 41.9100064) and (-87.7421459 <= float(x2) <= -87.50):
            return "Downtown"
        else:
            return "Not Downtown"
        
    def age_group(x):
        if  0.0 <= x <= 18.0:
            return "below 18"
        elif 18.0 < x <= 30.0:
            return "between 19 and 30"
        elif 30.0< x <= 40.0:
            return "between 31 and 40"
        elif 40.0< x <= 50.0:
            return "between 41 and 50"
        elif 50.0< x <= 60.0:
            return "between 51 and 60"
        else:
            return "Over 60"

        
    # Feature engineering
    train["INJURY"] = train["MOST_SEVERE_INJURY"].apply(lambda x: injury(x))
    train["FATALITIES"] = train["MOST_SEVERE_INJURY"].apply(lambda x: fatalities(x))
    train["AIRBAG_DEPLOYED"] = train["AIRBAG_DEPLOYED"].apply(lambda x: airbag(x))
    train["CRASH_HOUR"] = train["CRASH_HOUR"].apply(lambda x: crash_hour(int(x)))
    train["TRAFFICWAY_TYPE"] = train["TRAFFICWAY_TYPE"].apply(lambda x: traffic_way(x))
    train["FIRST_CONTACT_POINT"] = train["FIRST_CONTACT_POINT"].apply(lambda x: contact_point(x))
    train["MANEUVER"] = train["MANEUVER"].apply(lambda x: "TURN" if "TURN" in x else("LANE" if any(s in x for s in ["LANE","OVER","ENTER"]) else x))
    train["MANEUVER"] = train["MANEUVER"].apply(lambda x: "OTHER" if all(s not in x for s in ["AHEAD","TURN","UNKNOWN","LANE","BACKING"]) else x)
    train["SAFETY_EQUIPMENT"] = train["SAFETY_EQUIPMENT"].apply(lambda x: equip_used(x))
    train["AIRBAG_DEPLOYED"] = train["AIRBAG_DEPLOYED"].apply(lambda x: airbag(x))
    train["TRAFFIC_CONTROL_DEVICE"]= train["TRAFFIC_CONTROL_DEVICE"].apply(lambda x: traffic_control(x))
    train["Location"] = train.apply(lambda x: location(x["LATITUDE"], x["LONGITUDE"]), axis = 1)
    train["AGE_BIN"] = train["AGE"].apply(lambda x: age_group(x))
    
    train["VEHICLE_AGE"] = train["CRASH_DATE"].dt.year-train["VEHICLE_YEAR"]
    train.loc[train["VEHICLE_AGE"] < 0, "VEHICLE_AGE"] = 0
    train.drop(["VEHICLE_YEAR", "MOST_SEVERE_INJURY"], axis=1, inplace=True)  # "CRASH_DATE" for Tony
     
    # Return only train set for EDA purposes     
    return(train)

In [17]:
train = eda_preprocessor(data)

# Further Analysis

## Summary of Injuries Fatalities 


- **14% of fatalities** and **10% of injuries** happened to **people aged over 60**.
- **12% of fatalities** and **11% of injuries** happened in **vehicles aged over 15**.


- **25% of fatalities** and **13% of injuries** happened to **cyclers with no contrasting clothing**.
- **6% of injuries** happened to pedestrians or pedalists **while crossing the road**.
- **22% of fatalities** happened where drivers were **failed to reduce speed**.
- **22% of injuries** happened where drivers were **failed to yield right of way**.


- **8% of fatalities** happened where drivers were **too fast for conditions**.
- **16% of fatalities** and **35% of injuries** happened in which **cars were failed to deploy airbags**.
- **14% of fatalities** and **13% of injuries** happened to **drivers failed to use safety equipment**.


- **22% of fatalities** and **11% of injuries** happened to **pedestrians**.
- **5% of fatalities** and **1% of injuries** happened in **the excess of speed limits**.


- **55% of fatalities** and **33% of injuries** happened **at night accidents**.
- **11% of fatalities** and **16% of injuries** happened in **wet roads**.
- **39% of fatalities** and **47% of injuries** happened in **collisions straight ahead**.
- **5% of fatalities** and **12% of injuries** happened in **a turn maneuver**.

In [18]:
train = eda_preprocessor(data)

In [19]:
def injury_fatality_summarizer(col, df=train):
    x=df[df['INJURY']=='INJURED'][col].value_counts(normalize=True).reset_index(name='injury')
    y=df[df['FATALITIES']=='FATAL'][col].value_counts(normalize=True).reset_index(name='fatal')
    
    table=x.merge(y, how='inner', on=['index']).sort_values(['fatal','injury'], ascending=False)
    table=table.rename(columns={'index':col, 'injury':'PERCENTAGE OF INJURIES', 'fatal':'PERCENTAGE OF FATALITIES'})
    return(table[:12].round(2))

In [20]:
from IPython.core import display as ICD

for col in train.columns:
    table = injury_fatality_summarizer(col)
    ICD.display(table)

,CRASH_DATE,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,2018-04-10 18:20:00,0.0,0.20
1,2018-03-16 16:27:00,0.0,0.06
2,2018-03-23 20:44:00,0.0,0.06
3,2020-06-03 21:57:00,0.0,0.05
4,2019-01-27 01:20:00,0.0,0.05
5,2018-06-01 00:18:00,0.0,0.05
6,2020-03-11 13:10:00,0.0,0.03
7,2018-01-20 21:40:00,0.0,0.03
8,2018-09-23 17:20:00,0.0,0.03
9,2018-08-17 12:41:00,0.0,0.03


,POSTED_SPEED_LIMIT,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,30,0.78,0.66
1,35,0.10,0.30
3,40,0.02,0.03
2,25,0.05,0.02


,TRAFFIC_CONTROL_DEVICE,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
1,NO_SIGN,0.45,0.55
0,SIGN,0.55,0.45


,DEVICE_CONDITION,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,FUNCTIONING PROPERLY,0.50,0.5
1,NO CONTROLS,0.48,0.5


,WEATHER_CONDITION,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,CLEAR,0.82,0.80
1,RAIN,0.11,0.08
2,CLOUDY/OVERCAST,0.03,0.08
3,SNOW,0.03,0.05


...

,INJURY,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,INJURED,1.0,1.0


,FATALITIES,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,FATAL,0.01,1.0


,Location,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,Not Downtown,0.74,0.8
1,Downtown,0.26,0.2


,AGE_BIN,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
0,Over 60,0.30,0.41
1,between 19 and 30,0.25,0.27
2,between 31 and 40,0.16,0.16
5,below 18,0.05,0.08
3,between 41 and 50,0.12,0.06
4,between 51 and 60,0.11,0.03


,VEHICLE_AGE,PERCENTAGE OF INJURIES,PERCENTAGE OF FATALITIES
13,15.0,0.04,0.17
9,11.0,0.05,0.10
3,2.0,0.06,0.07
4,5.0,0.06,0.07
5,6.0,0.05,0.07
7,7.0,0.05,0.05
8,0.0,0.05,0.05
10,13.0,0.04,0.05
11,14.0,0.04,0.05
16,17.0,0.03,0.05


In [21]:
train.head()

,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_DIRECTION,DOORING_I,WORK_ZONE_I,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,LATITUDE,LONGITUDE,NUM_PASSENGERS,LIC_PLATE_STATE,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,FIRST_CONTACT_POINT,PERSON_TYPE,CITY,SEX,AGE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,BAC_RESULT VALUE,CELL_PHONE_USE,TOTAL_FATAL,INJURY,FATALITIES,Location,AGE_BIN,VEHICLE_AGE
29244,2018-02-05 13:30:00,30,SIGN,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,ANGLE,DIVIDED,4.0,STRAIGHT ON GRADE,OTHER,CLEAR,Y,N,N,DISREGARDING TRAFFIC SIGNALS,FAILING TO YIELD RIGHT-OF-WAY,E,N,N,2,0,0,0,Afternoon,41.891233581,-87.613501254,0,IL,OTHER,SPORT UTILITY VEHICLE (SUV),PERSONAL,S,STRAIGHT AHEAD,1,N,SIDE,DRIVER,CHICAGO,M,26.0,USED SAFETY EQUIP,DEPLOYED,NONE,NONE,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Downtown,between 19 and 30,1.0
22376,2018-08-07 14:55:00,30,SIGN,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT_DIVIDED,3.0,STRAIGHT AND LEVEL,WET,NO DEFECTS,N,N,N,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,S,N,N,2,0,0,0,Afternoon,41.879610255,-87.63097225,0,IL,NONE,PASSENGER,TAXI/FOR HIRE,S,LANE,1,N,SIDE,DRIVER,HOFFMAN ESTATES,M,51.0,UNABLE TO DETERMINE,NOT DEPLOYED,NONE,IMPROPER LANE CHANGE,OTHER,UNABLE TO DETERMINE,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Downtown,between 51 and 60,NaN
49790,2019-05-10 12:00:00,30,NO_SIGN,NO CONTROLS,CLEAR,DAYLIGHT,OTHER OBJECT,NOT_DIVIDED,2.0,STRAIGHT AND LEVEL,DRY,"RUT, HOLES",N,N,N,ROAD CONSTRUCTION/MAINTENANCE,NOT APPLICABLE,N,N,N,1,0,0,0,Afternoon,41.966268686,-87.836830283,0,IL,NONE,PASSENGER,PERSONAL,S,STRAIGHT AHEAD,1,N,OTHER,DRIVER,CHICAGO,M,73.0,USED SAFETY EQUIP,NOT DEPLOYED,NONE,NONE,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Not Downtown,Over 60,0.0
32552,2018-01-13 11:12:00,30,NO_SIGN,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED,2.0,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,N,N,UNABLE TO DETERMINE,NOT APPLICABLE,W,N,N,2,0,0,0,Morning,41.92902255,-87.659229555,0,IL,NONE,TRUCK - SINGLE UNIT,UNKNOWN/NA,E,TURN,1,N,OTHER,DRIVER,OTHER,F,NaN,UNABLE TO DETERMINE,NOT APPLICABLE,NONE,NONE,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Not Downtown,Over 60,NaN
1957,2020-07-18 02:00:00,30,SIGN,FUNCTIONING PROPERLY,CLEAR,DARKNESS,TURNING,DIVIDED,2.0,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,N,Y,IMPROPER TURNING/NO SIGNAL,UNABLE TO DETERMINE,S,N,N,2,0,0,0,Early_morning,41.707182175,-87.620361492,0,IL,NONE,PASSENGER,PERSONAL,E,TURN,1,N,OTHER,DRIVER,CHICAGO,F,30.0,USED SAFETY EQUIP,NOT DEPLOYED,NONE,OTHER,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Not Downtown,between 19 and 30,12.0


In [22]:
def injury_summarizer(col1, col2, df=train):
    x = train[train['INJURY'] == 'INJURED'].groupby([col1, col2])['INJURY'].value_counts().unstack()
    x = x.apply(lambda x: x/x.sum(), axis=0).round(2)
    
    return x

In [23]:
def fatal_summarizer(col1, col2, df=train):
    y = train[train['FATALITIES'] == 'FATAL'].groupby([col1, col2])['FATALITIES'].value_counts().unstack()
    y = y.apply(lambda x: x/x.sum(), axis=0).round(2)
    
    return y

In [24]:
# Test function
injury_summarizer("SEX", "AGE_BIN")

INJURY                                 INJURED
SEX                 AGE_BIN                   
F                   Over 60               0.06
                    below 18              0.02
                    between 19 and 30     0.11
                    between 31 and 40     0.07
                    between 41 and 50     0.04
                    between 51 and 60     0.04
M                   Over 60               0.12
                    below 18              0.03
                    between 19 and 30     0.14
                    between 31 and 40     0.09
                    between 41 and 50     0.08
                    between 51 and 60     0.07
UNABLE TO DETERMINE Over 60               0.08
                    below 18              0.00
                    between 19 and 30     0.00
X                   Over 60               0.04
                    between 19 and 30     0.00

In [25]:
train.head()

,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_DIRECTION,DOORING_I,WORK_ZONE_I,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,LATITUDE,LONGITUDE,NUM_PASSENGERS,LIC_PLATE_STATE,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,FIRST_CONTACT_POINT,PERSON_TYPE,CITY,SEX,AGE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,BAC_RESULT VALUE,CELL_PHONE_USE,TOTAL_FATAL,INJURY,FATALITIES,Location,AGE_BIN,VEHICLE_AGE
29244,2018-02-05 13:30:00,30,SIGN,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,ANGLE,DIVIDED,4.0,STRAIGHT ON GRADE,OTHER,CLEAR,Y,N,N,DISREGARDING TRAFFIC SIGNALS,FAILING TO YIELD RIGHT-OF-WAY,E,N,N,2,0,0,0,Afternoon,41.891233581,-87.613501254,0,IL,OTHER,SPORT UTILITY VEHICLE (SUV),PERSONAL,S,STRAIGHT AHEAD,1,N,SIDE,DRIVER,CHICAGO,M,26.0,USED SAFETY EQUIP,DEPLOYED,NONE,NONE,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Downtown,between 19 and 30,1.0
22376,2018-08-07 14:55:00,30,SIGN,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT_DIVIDED,3.0,STRAIGHT AND LEVEL,WET,NO DEFECTS,N,N,N,IMPROPER OVERTAKING/PASSING,UNABLE TO DETERMINE,S,N,N,2,0,0,0,Afternoon,41.879610255,-87.63097225,0,IL,NONE,PASSENGER,TAXI/FOR HIRE,S,LANE,1,N,SIDE,DRIVER,HOFFMAN ESTATES,M,51.0,UNABLE TO DETERMINE,NOT DEPLOYED,NONE,IMPROPER LANE CHANGE,OTHER,UNABLE TO DETERMINE,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Downtown,between 51 and 60,NaN
49790,2019-05-10 12:00:00,30,NO_SIGN,NO CONTROLS,CLEAR,DAYLIGHT,OTHER OBJECT,NOT_DIVIDED,2.0,STRAIGHT AND LEVEL,DRY,"RUT, HOLES",N,N,N,ROAD CONSTRUCTION/MAINTENANCE,NOT APPLICABLE,N,N,N,1,0,0,0,Afternoon,41.966268686,-87.836830283,0,IL,NONE,PASSENGER,PERSONAL,S,STRAIGHT AHEAD,1,N,OTHER,DRIVER,CHICAGO,M,73.0,USED SAFETY EQUIP,NOT DEPLOYED,NONE,NONE,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Not Downtown,Over 60,0.0
32552,2018-01-13 11:12:00,30,NO_SIGN,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED,2.0,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,N,N,UNABLE TO DETERMINE,NOT APPLICABLE,W,N,N,2,0,0,0,Morning,41.92902255,-87.659229555,0,IL,NONE,TRUCK - SINGLE UNIT,UNKNOWN/NA,E,TURN,1,N,OTHER,DRIVER,OTHER,F,NaN,UNABLE TO DETERMINE,NOT APPLICABLE,NONE,NONE,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Not Downtown,Over 60,NaN
1957,2020-07-18 02:00:00,30,SIGN,FUNCTIONING PROPERLY,CLEAR,DARKNESS,TURNING,DIVIDED,2.0,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,N,Y,IMPROPER TURNING/NO SIGNAL,UNABLE TO DETERMINE,S,N,N,2,0,0,0,Early_morning,41.707182175,-87.620361492,0,IL,NONE,PASSENGER,PERSONAL,E,TURN,1,N,OTHER,DRIVER,CHICAGO,F,30.0,USED SAFETY EQUIP,NOT DEPLOYED,NONE,OTHER,NOT OBSCURED,NORMAL,UNABLE TO DETERMINE,UNABLE TO DETERMINE,0,UNABLE TO DETERMINE,0,NOT INJURED,NOT FATAL,Not Downtown,between 19 and 30,12.0


### Multivariate for injuries

In [26]:
from IPython.core import display as ICD

train_col = ['AGE_BIN', 'PHYSICAL_CONDITION', 'DRIVER_VISION', 'SEX', 'CRASH_HOUR', 'PRIM_CONTRIBUTORY_CAUSE',
            'INTERSECTION_RELATED_I', 'ROAD_DEFECT', 'LIGHTING_CONDITION', 'WEATHER_CONDITION', 'POSTED_SPEED_LIMIT']

for col1 in train_col:
    for col2 in train_col:
        if col1 != col2:
            table = injury_summarizer(col1, col2)
            ICD.display(table)

INJURY                                 INJURED
AGE_BIN           PHYSICAL_CONDITION          
Over 60           EMOTIONAL               0.00
                  FATIGUED/ASLEEP         0.00
                  HAD BEEN DRINKING       0.00
                  ILLNESS/FAINTED         0.00
                  IMPAIRED - ALCOHOL      0.00
                  NORMAL                  0.10
                  OTHER                   0.00
                  REMOVED BY EMS          0.01
                  UNABLE TO DETERMINE     0.20
below 18          EMOTIONAL               0.00
                  IMPAIRED - ALCOHOL      0.00
                  NORMAL                  0.04
                  OTHER                   0.00
                  REMOVED BY EMS          0.00
                  UNABLE TO DETERMINE     0.01
between 19 and 30 EMOTIONAL               0.00
                  FATIGUED/ASLEEP         0.00
                  HAD BEEN DRINKING       0.00
                  ILLNESS/FAINTED         0.00
                  IMPAIRED - ALCOHOL      0.00
                  IMPAIRED - DRUGS        0.00
                  NORMAL                  0.20
                  OTHER                   0.00
                  REMOVED BY EMS          0.01
                  UNABLE TO DETERMINE     0.03
between 31 and 40 EMOTIONAL               0.00
                  FATIGUED/ASLEEP         0.00
                  ILLNESS/FAINTED         0.00
                  IMPAIRED - ALCOHOL      0.00
                  IMPAIRED - DRUGS        0.00
                  MEDICATED               0.00
                  NORMAL                  0.13
                  OTHER                   0.00
                  REMOVED BY EMS          0.01
                  UNABLE TO DETERMINE     0.02
between 41 and 50 EMOTIONAL               0.00
                  FATIGUED/ASLEEP         0.00
                  HAD BEEN DRINKING       0.00
                  ILLNESS/FAINTED         0.00
                  IMPAIRED - ALCOHOL      0.00
                  MEDICATED               0.00
                  NORMAL                  0.10
                  OTHER                   0.00
                  REMOVED BY EMS          0.00
                  UNABLE TO DETERMINE     0.02
between 51 and 60 EMOTIONAL               0.00
                  FATIGUED/ASLEEP         0.00
                  HAD BEEN DRINKING       0.00
                  ILLNESS/FAINTED         0.00
                  IMPAIRED - ALCOHOL      0.00
                  IMPAIRED - DRUGS        0.00
                  MEDICATED               0.00
                  NORMAL                  0.09
                  OTHER                   0.00
                  REMOVED BY EMS          0.00
                  UNABLE TO DETERMINE     0.01

INJURY                                    INJURED
AGE_BIN           DRIVER_VISION                  
Over 60           BLINDED - SUNLIGHT         0.00
                  BLOWING MATERIALS          0.00
                  MOVING VEHICLES            0.00
                  NOT OBSCURED               0.08
                  OTHER                      0.23
                  PARKED VEHICLES            0.00
                  TREES, PLANTS              0.00
                  WINDSHIELD (WATER/ICE)     0.00
below 18          BLINDED - SUNLIGHT         0.00
                  MOVING VEHICLES            0.00
                  NOT OBSCURED               0.03
                  OTHER                      0.02
                  PARKED VEHICLES            0.00
between 19 and 30 BLINDED - SUNLIGHT         0.00
                  MOVING VEHICLES            0.00
                  NOT OBSCURED               0.16
                  OTHER                      0.09
                  PARKED VEHICLES            0.00
                  WINDSHIELD (WATER/ICE)     0.00
between 31 and 40 BLINDED - SUNLIGHT         0.00
                  BUILDINGS                  0.00
                  MOVING VEHICLES            0.00
                  NOT OBSCURED               0.10
                  OTHER                      0.06
                  PARKED VEHICLES            0.00
                  TREES, PLANTS              0.00
                  WINDSHIELD (WATER/ICE)     0.00
between 41 and 50 BLINDED - SUNLIGHT         0.00
                  MOVING VEHICLES            0.00
                  NOT OBSCURED               0.08
                  OTHER                      0.04
                  PARKED VEHICLES            0.00
                  WINDSHIELD (WATER/ICE)     0.00
between 51 and 60 BLINDED - SUNLIGHT         0.00
                  BUILDINGS                  0.00
                  MOVING VEHICLES            0.00
                  NOT OBSCURED               0.07
                  OTHER                      0.04
                  WINDSHIELD (WATER/ICE)     0.00

INJURY                                 INJURED
AGE_BIN           SEX                         
Over 60           F                       0.06
                  M                       0.12
                  UNABLE TO DETERMINE     0.08
                  X                       0.04
below 18          F                       0.02
                  M                       0.03
                  UNABLE TO DETERMINE     0.00
between 19 and 30 F                       0.11
                  M                       0.14
                  UNABLE TO DETERMINE     0.00
                  X                       0.00
between 31 and 40 F                       0.07
                  M                       0.09
between 41 and 50 F                       0.04
                  M                       0.08
between 51 and 60 F                       0.04
                  M                       0.07

INJURY                           INJURED
AGE_BIN           CRASH_HOUR            
Over 60           Afternoon         0.11
                  Early_morning     0.04
                  Morning           0.05
                  Night             0.11
below 18          Afternoon         0.02
                  Early_morning     0.01
                  Morning           0.01
                  Night             0.01
between 19 and 30 Afternoon         0.08
                  Early_morning     0.03
                  Morning           0.04
                  Night             0.09
between 31 and 40 Afternoon         0.06
                  Early_morning     0.02
                  Morning           0.03
                  Night             0.05
between 41 and 50 Afternoon         0.05
                  Early_morning     0.01
                  Morning           0.02
                  Night             0.04
between 51 and 60 Afternoon         0.05
                  Early_morning     0.01
                  Morning           0.02
                  Night             0.03

INJURY                                                                INJURED
AGE_BIN           PRIM_CONTRIBUTORY_CAUSE                                    
Over 60           ANIMAL                                                 0.00
                  BICYCLE ADVANCING LEGALLY ON RED LIGHT                 0.00
                  CELL PHONE USE OTHER THAN TEXTING                      0.00
                  DISREGARDING OTHER TRAFFIC SIGNS                       0.00
                  DISREGARDING ROAD MARKINGS                             0.00
...                                                                       ...
between 51 and 60 TURNING RIGHT ON RED                                   0.00
                  UNABLE TO DETERMINE                                    0.03
                  UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...     0.00
                  VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...     0.00
                  WEATHER                                                0.00

[194 rows x 1 columns]

...

INJURY                                                  INJURED
POSTED_SPEED_LIMIT PRIM_CONTRIBUTORY_CAUSE                     
0                  DRIVING SKILLS/KNOWLEDGE/EXPERIENCE      0.0
                   FAILING TO YIELD RIGHT-OF-WAY            0.0
                   FOLLOWING TOO CLOSELY                    0.0
                   IMPROPER BACKING                         0.0
                   IMPROPER LANE USAGE                      0.0
...                                                         ...
45                 IMPROPER LANE USAGE                      0.0
                   UNABLE TO DETERMINE                      0.0
55                 IMPROPER TURNING/NO SIGNAL               0.0
                   UNABLE TO DETERMINE                      0.0
                   WEATHER                                  0.0

[170 rows x 1 columns]

INJURY                                     INJURED
POSTED_SPEED_LIMIT INTERSECTION_RELATED_I         
0                  N                          0.00
                   Y                          0.00
3                  N                          0.00
5                  N                          0.00
10                 N                          0.00
                   Y                          0.00
15                 N                          0.01
                   Y                          0.00
20                 N                          0.01
                   Y                          0.01
25                 N                          0.04
                   Y                          0.02
30                 N                          0.43
                   Y                          0.35
32                 Y                          0.00
35                 N                          0.06
                   Y                          0.04
40                 N                          0.01
                   Y                          0.00
45                 N                          0.00
                   Y                          0.00
55                 N                          0.00
                   Y                          0.00

INJURY                                INJURED
POSTED_SPEED_LIMIT ROAD_DEFECT               
0                  CLEAR                 0.00
                   NO DEFECTS            0.00
                   OTHER                 0.00
3                  CLEAR                 0.00
                   NO DEFECTS            0.00
5                  NO DEFECTS            0.00
10                 CLEAR                 0.00
                   NO DEFECTS            0.00
                   OTHER                 0.00
15                 CLEAR                 0.00
                   NO DEFECTS            0.01
                   OTHER                 0.00
20                 CLEAR                 0.00
                   NO DEFECTS            0.02
                   OTHER                 0.00
                   RUT, HOLES            0.00
25                 CLEAR                 0.01
                   DEBRIS ON ROADWAY     0.00
                   NO DEFECTS            0.05
                   RUT, HOLES            0.00
30                 CLEAR                 0.09
                   DEBRIS ON ROADWAY     0.00
                   NO DEFECTS            0.67
                   OTHER                 0.01
                   RUT, HOLES            0.00
                   SHOULDER DEFECT       0.00
                   WORN SURFACE          0.00
32                 NO DEFECTS            0.00
35                 CLEAR                 0.01
                   NO DEFECTS            0.09
                   OTHER                 0.00
40                 CLEAR                 0.00
                   NO DEFECTS            0.02
                   OTHER                 0.00
45                 CLEAR                 0.00
                   NO DEFECTS            0.00
55                 NO DEFECTS            0.00

INJURY                                     INJURED
POSTED_SPEED_LIMIT LIGHTING_CONDITION             
0                  DARKNESS, LIGHTED ROAD     0.00
                   DAYLIGHT                   0.00
3                  DAYLIGHT                   0.00
5                  DARKNESS, LIGHTED ROAD     0.00
                   DAYLIGHT                   0.00
                   DUSK                       0.00
10                 DARKNESS, LIGHTED ROAD     0.00
                   DAYLIGHT                   0.00
                   DUSK                       0.00
15                 DARKNESS                   0.00
                   DARKNESS, LIGHTED ROAD     0.00
                   DAWN                       0.00
                   DAYLIGHT                   0.01
                   DUSK                       0.00
20                 DARKNESS                   0.00
                   DARKNESS, LIGHTED ROAD     0.00
                   DAYLIGHT                   0.01
                   DUSK                       0.00
25                 DARKNESS                   0.00
                   DARKNESS, LIGHTED ROAD     0.01
                   DAWN                       0.00
                   DAYLIGHT                   0.04
                   DUSK                       0.00
30                 DARKNESS                   0.04
                   DARKNESS, LIGHTED ROAD     0.23
                   DAWN                       0.01
                   DAYLIGHT                   0.48
                   DUSK                       0.02
32                 DAYLIGHT                   0.00
35                 DARKNESS                   0.01
                   DARKNESS, LIGHTED ROAD     0.03
                   DAWN                       0.00
                   DAYLIGHT                   0.06
                   DUSK                       0.00
40                 DARKNESS                   0.00
                   DARKNESS, LIGHTED ROAD     0.00
                   DAWN                       0.00
                   DAYLIGHT                   0.01
                   DUSK                       0.00
45                 DARKNESS                   0.00
                   DARKNESS, LIGHTED ROAD     0.00
                   DAWN                       0.00
                   DAYLIGHT                   0.00
55                 DARKNESS, LIGHTED ROAD     0.00
                   DAYLIGHT                   0.00

INJURY                                    INJURED
POSTED_SPEED_LIMIT WEATHER_CONDITION             
0                  CLEAR                     0.00
                   CLOUDY/OVERCAST           0.00
                   RAIN                      0.00
                   SNOW                      0.00
3                  CLEAR                     0.00
                   RAIN                      0.00
5                  CLEAR                     0.00
10                 CLEAR                     0.00
                   CLOUDY/OVERCAST           0.00
                   RAIN                      0.00
15                 CLEAR                     0.01
                   CLOUDY/OVERCAST           0.00
                   RAIN                      0.00
                   SNOW                      0.00
20                 CLEAR                     0.02
                   CLOUDY/OVERCAST           0.00
                   RAIN                      0.00
                   SNOW                      0.00
25                 CLEAR                     0.04
                   CLOUDY/OVERCAST           0.00
                   FOG/SMOKE/HAZE            0.00
                   RAIN                      0.01
                   SNOW                      0.00
30                 BLOWING SNOW              0.00
                   CLEAR                     0.64
                   CLOUDY/OVERCAST           0.02
                   FOG/SMOKE/HAZE            0.00
                   FREEZING RAIN/DRIZZLE     0.00
                   OTHER                     0.00
                   RAIN                      0.08
                   SLEET/HAIL                0.00
                   SNOW                      0.02
32                 CLEAR                     0.00
35                 CLEAR                     0.08
                   CLOUDY/OVERCAST           0.00
                   FOG/SMOKE/HAZE            0.00
                   OTHER                     0.00
                   RAIN                      0.01
                   SLEET/HAIL                0.00
                   SNOW                      0.00
40                 CLEAR                     0.01
                   CLOUDY/OVERCAST           0.00
                   FOG/SMOKE/HAZE            0.00
                   RAIN                      0.00
45                 CLEAR                     0.00
                   RAIN                      0.00
55                 CLEAR                     0.00

**Insights from multivariate analysis**

- AGE_BIN: Over 60 = 0.30, between 19 and 30 = 0.25
- AGE_BIN & SEX: Over 60 & M = 0.12, between 19 and 30 & M = 0.14
- AGE_BIN & CRASH_HOUR: Over 60 & Night	= 0.11
- AGE_BIN & DARKNESS: 0.11 (Much more than other groups)
- PHYSICAL_CONDITION & SEX: IMPAIRED - ALCOHOL & MALE = 0.01
- PHYSICAL_CONDITION & WEATHER_CONDITION: NORMAL & RAIN = 0.07
- CRASH_HOUR: Afternoon	= 0.37, Night = 0.33
- CRASH_HOUR & LIGHTING_CONDITION: Night & DARKNESS, LIGHTED ROAD = 0.21

**Further consider**
- AGE_BIN & PRIM_CAUSE: None
- SEX & PRIM_CAUSE: Female & FAILING TO YIELD RIGHT-OF-WAY = 0.08

In [29]:
# AGE_BIN & PRIM_CAUSE
pd.set_option("max_rows", None)
injury_summarizer('AGE_BIN', 'PRIM_CONTRIBUTORY_CAUSE')

INJURY                                                                INJURED
AGE_BIN           PRIM_CONTRIBUTORY_CAUSE                                    
Over 60           ANIMAL                                                 0.00
                  BICYCLE ADVANCING LEGALLY ON RED LIGHT                 0.00
                  CELL PHONE USE OTHER THAN TEXTING                      0.00
                  DISREGARDING OTHER TRAFFIC SIGNS                       0.00
                  DISREGARDING ROAD MARKINGS                             0.00
                  DISREGARDING STOP SIGN                                 0.01
                  DISREGARDING TRAFFIC SIGNALS                           0.01
                  DISREGARDING YIELD SIGN                                0.00
                  DISTRACTION - FROM INSIDE VEHICLE                      0.00
                  DISTRACTION - FROM OUTSIDE VEHICLE                     0.00
                  DRIVING ON WRONG SIDE/WRONG WAY                        0.00
                  DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                    0.01
                  EQUIPMENT - VEHICLE CONDITION                          0.00
                  EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST      0.00
                  EXCEEDING AUTHORIZED SPEED LIMIT                       0.00
                  EXCEEDING SAFE SPEED FOR CONDITIONS                    0.00
                  FAILING TO REDUCE SPEED TO AVOID CRASH                 0.03
                  FAILING TO YIELD RIGHT-OF-WAY                          0.05
                  FOLLOWING TOO CLOSELY                                  0.02
                  HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)        0.00
                  IMPROPER BACKING                                       0.00
                  IMPROPER LANE USAGE                                    0.01
                  IMPROPER OVERTAKING/PASSING                            0.01
                  IMPROPER TURNING/NO SIGNAL                             0.01
                  MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT              0.00
                  NOT APPLICABLE                                         0.01
                  OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...     0.01
                  PHYSICAL CONDITION OF DRIVER                           0.01
                  RELATED TO BUS STOP                                    0.00
                  ROAD CONSTRUCTION/MAINTENANCE                          0.00
                  ROAD ENGINEERING/SURFACE/MARKING DEFECTS               0.00
                  TURNING RIGHT ON RED                                   0.00
                  UNABLE TO DETERMINE                                    0.09
                  UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...     0.00
                  VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...     0.00
                  WEATHER                                                0.00
below 18          ANIMAL                                                 0.00
                  CELL PHONE USE OTHER THAN TEXTING                      0.00
                  DISREGARDING OTHER TRAFFIC SIGNS                       0.00
                  DISREGARDING STOP SIGN                                 0.00
                  DISREGARDING TRAFFIC SIGNALS                           0.00
                  DISTRACTION - FROM OUTSIDE VEHICLE                     0.00
                  DRIVING ON WRONG SIDE/WRONG WAY                        0.00
                  DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                    0.00
                  EQUIPMENT - VEHICLE CONDITION                          0.00
                  EXCEEDING AUTHORIZED SPEED LIMIT                       0.00
                  FAILING TO REDUCE SPEED TO AVOID CRASH                 0.00
                  FAILING TO YIELD RIGHT-OF-WAY                          0.01
                  FOLLOWING TOO CLOSELY                                  0.00
                  HAD 

In [31]:
# SEX & PRIM_CAUSE
pd.set_option("max_rows", None)
injury_summarizer('SEX', 'PRIM_CONTRIBUTORY_CAUSE')

INJURY                                                                  INJURED
SEX                 PRIM_CONTRIBUTORY_CAUSE                                    
F                   ANIMAL                                                 0.00
                    CELL PHONE USE OTHER THAN TEXTING                      0.00
                    DISREGARDING OTHER TRAFFIC SIGNS                       0.00
                    DISREGARDING ROAD MARKINGS                             0.00
                    DISREGARDING STOP SIGN                                 0.01
                    DISREGARDING TRAFFIC SIGNALS                           0.02
                    DISREGARDING YIELD SIGN                                0.00
                    DISTRACTION - FROM INSIDE VEHICLE                      0.00
                    DISTRACTION - FROM OUTSIDE VEHICLE                     0.00
                    DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATI...     0.00
                    DRIVING ON WRONG SIDE/WRONG WAY                        0.00
                    DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                    0.01
                    EQUIPMENT - VEHICLE CONDITION                          0.00
                    EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST      0.00
                    EXCEEDING AUTHORIZED SPEED LIMIT                       0.00
                    EXCEEDING SAFE SPEED FOR CONDITIONS                    0.00
                    FAILING TO REDUCE SPEED TO AVOID CRASH                 0.03
                    FAILING TO YIELD RIGHT-OF-WAY                          0.08
                    FOLLOWING TOO CLOSELY                                  0.03
                    HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)        0.00
                    IMPROPER BACKING                                       0.00
                    IMPROPER LANE USAGE                                    0.01
                    IMPROPER OVERTAKING/PASSING                            0.01
                    IMPROPER TURNING/NO SIGNAL                             0.01
                    NOT APPLICABLE                                         0.01
                    OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...     0.01
                    PHYSICAL CONDITION OF DRIVER                           0.01
                    RELATED TO BUS STOP                                    0.00
                    ROAD CONSTRUCTION/MAINTENANCE                          0.00
                    ROAD ENGINEERING/SURFACE/MARKING DEFECTS               0.00
                    TURNING RIGHT ON RED                                   0.00
                    UNABLE TO DETERMINE                                    0.08
                    UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...     0.00
                    VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...     0.00
                    WEATHER                                                0.01
M                   ANIMAL                                                 0.00
                    BICYCLE ADVANCING LEGALLY ON RED LIGHT                 0.00
                    CELL PHONE USE OTHER THAN TEXTING                      0.00
                    DISREGARDING OTHER TRAFFIC SIGNS                       0.00
                    DISREGARDING ROAD MARKINGS                             0.00
                    DISREGARDING STOP SIGN                                 0.01
                    DISREGARDING TRAFFIC SIGNALS                           0.04
                    DISREGARDING YIELD SIGN                                0.00
                    DISTRACTION - FROM INSIDE VEHICLE                      0.00
                    DISTRACTION - FROM OUTSIDE VEHICLE                     0.00
                    DRIVING ON WRONG SIDE/WRONG WAY                        0.01
                    DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                    0.01
                    EQUIPMENT - VEHICLE CONDITION                          0.00


### Multivariate for fatal injuries

In [28]:
for col1 in train_col:
    for col2 in train_col:
        if col1 != col2:
            table = fatal_summarizer(col1, col2)
            ICD.display(table)

FATALITIES                             FATAL
AGE_BIN           PHYSICAL_CONDITION        
Over 60           NORMAL                0.12
                  REMOVED BY EMS        0.03
                  UNABLE TO DETERMINE   0.25
below 18          NORMAL                0.08
between 19 and 30 EMOTIONAL             0.02
                  NORMAL                0.14
                  OTHER                 0.03
                  UNABLE TO DETERMINE   0.08
between 31 and 40 NORMAL                0.05
                  UNABLE TO DETERMINE   0.11
between 41 and 50 REMOVED BY EMS        0.02
                  UNABLE TO DETERMINE   0.05
between 51 and 60 NORMAL                0.02
                  UNABLE TO DETERMINE   0.02

FATALITIES                         FATAL
AGE_BIN           DRIVER_VISION         
Over 60           NOT OBSCURED      0.05
                  OTHER             0.36
below 18          NOT OBSCURED      0.02
                  OTHER             0.03
                  PARKED VEHICLES   0.03
between 19 and 30 NOT OBSCURED      0.11
                  OTHER             0.16
between 31 and 40 MOVING VEHICLES   0.02
                  NOT OBSCURED      0.02
                  OTHER             0.12
between 41 and 50 NOT OBSCURED      0.02
                  OTHER             0.05
between 51 and 60 NOT OBSCURED      0.02
                  OTHER             0.02

FATALITIES                             FATAL
AGE_BIN           SEX                       
Over 60           F                     0.09
                  M                     0.12
                  UNABLE TO DETERMINE   0.11
                  X                     0.08
below 18          F                     0.03
                  M                     0.05
between 19 and 30 F                     0.08
                  M                     0.19
between 31 and 40 F                     0.08
                  M                     0.08
between 41 and 50 F                     0.02
                  M                     0.05
between 51 and 60 M                     0.03

FATALITIES                       FATAL
AGE_BIN           CRASH_HOUR          
Over 60           Afternoon       0.09
                  Early_morning   0.08
                  Morning         0.03
                  Night           0.20
below 18          Afternoon       0.05
                  Night           0.03
between 19 and 30 Afternoon       0.03
                  Early_morning   0.02
                  Morning         0.02
                  Night           0.20
between 31 and 40 Afternoon       0.02
                  Early_morning   0.03
                  Morning         0.03
                  Night           0.08
between 41 and 50 Afternoon       0.02
                  Morning         0.03
                  Night           0.02
between 51 and 60 Afternoon       0.02
                  Night           0.02

FATALITIES                                                            FATAL
AGE_BIN           PRIM_CONTRIBUTORY_CAUSE                                  
Over 60           DRIVING ON WRONG SIDE/WRONG WAY                      0.02
                  EXCEEDING AUTHORIZED SPEED LIMIT                     0.02
                  EXCEEDING SAFE SPEED FOR CONDITIONS                  0.02
                  FAILING TO REDUCE SPEED TO AVOID CRASH               0.06
                  IMPROPER LANE USAGE                                  0.03
                  OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...   0.03
                  PHYSICAL CONDITION OF DRIVER                         0.11
                  UNABLE TO DETERMINE                                  0.12
below 18          FAILING TO REDUCE SPEED TO AVOID CRASH               0.03
                  FAILING TO YIELD RIGHT-OF-WAY                        0.03
                  OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...   0.02
between 19 and 30 DISREGARDING TRAFFIC SIGNALS                         0.02
                  DISTRACTION - FROM INSIDE VEHICLE                    0.02
                  FAILING TO REDUCE SPEED TO AVOID CRASH               0.11
                  FAILING TO YIELD RIGHT-OF-WAY                        0.02
                  OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...   0.02
                  PHYSICAL CONDITION OF DRIVER                         0.05
                  UNABLE TO DETERMINE                                  0.05
between 31 and 40 FAILING TO YIELD RIGHT-OF-WAY                        0.02
                  OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...   0.02
                  UNABLE TO DETERMINE                                  0.12
between 41 and 50 FAILING TO YIELD RIGHT-OF-WAY                        0.02
                  UNABLE TO DETERMINE                                  0.05
between 51 and 60 DISREGARDING TRAFFIC SIGNALS                         0.02
                  FAILING TO REDUCE SPEED TO AVOID CRASH               0.02

...

FATALITIES                                                             FATAL
POSTED_SPEED_LIMIT PRIM_CONTRIBUTORY_CAUSE                                  
25                 FAILING TO YIELD RIGHT-OF-WAY                        0.02
30                 DISREGARDING TRAFFIC SIGNALS                         0.03
                   DISTRACTION - FROM INSIDE VEHICLE                    0.02
                   DRIVING ON WRONG SIDE/WRONG WAY                      0.02
                   EXCEEDING AUTHORIZED SPEED LIMIT                     0.02
                   EXCEEDING SAFE SPEED FOR CONDITIONS                  0.02
                   FAILING TO REDUCE SPEED TO AVOID CRASH               0.02
                   FAILING TO YIELD RIGHT-OF-WAY                        0.06
                   OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...   0.08
                   PHYSICAL CONDITION OF DRIVER                         0.16
                   UNABLE TO DETERMINE                                  0.25
35                 FAILING TO REDUCE SPEED TO AVOID CRASH               0.20
                   UNABLE TO DETERMINE                                  0.09
40                 IMPROPER LANE USAGE                                  0.03

FATALITIES                                 FATAL
POSTED_SPEED_LIMIT INTERSECTION_RELATED_I       
25                 N                        0.02
30                 N                        0.36
                   Y                        0.30
35                 N                        0.06
                   Y                        0.23
40                 N                        0.03

FATALITIES                      FATAL
POSTED_SPEED_LIMIT ROAD_DEFECT       
25                 NO DEFECTS    0.02
30                 CLEAR         0.14
                   NO DEFECTS    0.52
35                 NO DEFECTS    0.30
40                 NO DEFECTS    0.03

FATALITIES                                 FATAL
POSTED_SPEED_LIMIT LIGHTING_CONDITION           
25                 DARKNESS, LIGHTED ROAD   0.02
30                 DARKNESS                 0.02
                   DARKNESS, LIGHTED ROAD   0.34
                   DAYLIGHT                 0.30
35                 DARKNESS                 0.03
                   DAYLIGHT                 0.27
40                 DARKNESS, LIGHTED ROAD   0.03

FATALITIES                            FATAL
POSTED_SPEED_LIMIT WEATHER_CONDITION       
25                 CLEAR               0.02
30                 CLEAR               0.48
                   CLOUDY/OVERCAST     0.05
                   RAIN                0.08
                   SNOW                0.05
35                 CLEAR               0.27
                   CLOUDY/OVERCAST     0.03
40                 CLEAR               0.03

**Insights from multivariate analysis**

- AGE_BIN & CRASH_HOUR: Over 60 & Night = 0.20
- AGE_BIN & PRIM_CONTRIBUTORY_CAUSE: Over 60 & PHYSICAL CONDITION OF DRIVER = 0.11, Over 60 & FAILING TO REDUCE SPEED TO AVOID CRASH = 0.06, between 19 and 30 & FAILING TO REDUCE SPEED TO AVOID CRASH = 0.11, 
- PHYSICAL_CONDITION & CRASH_HOUR: NORMAL & Night = 0.28, EMOTIONAL & Morning = 0.02
- PHYSICAL_CONDITION & PRIM_CONTRIBUTORY_CAUSE: EMOTIONAL & DISTRACTION - FROM INSIDE VEHICLE = 0.02, NORMAL & FAILING TO REDUCE SPEED TO AVOID CRASH = 0.22
- PHYSICAL_CONDITION & POSTED_SPEED_LIMIT: NORMAL & Increasing of speed = 25:0.02, 30:0.16, 35:0.23
- DRIVER_VISION	& AGE_BIN: PARKED VEHICLES & below 18 = 0.03
- SEX & AGE_BIN: F & Over 60 = 0.09, while M & between 19 and 30 = 0.19
- SEX & CRASH_HOUR: F & Afternoon = 0.02, while M & Afternoon = 0.14
- SEX & PRIM_CONTRIBUTORY_CAUSE: F & FAILING TO REDUCE SPEED TO AVOID CRASH = 0.17, M & OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER = 0.06, M & FAILING TO YIELD RIGHT-OF-WAY = 0.06
- SEX & LIGHTING_CONDITION: M & DARKNESS, LIGHTED ROAD = 0.23, while F & DARKNESS, LIGHTED ROAD = 0.06
- CRASH_HOUR & PRIM_CONTRIBUTORY_CAUSE: Afternoon & PHYSICAL CONDITION OF DRIVER or FAILING TO YIELD RIGHT-OF-WAY = 0.06, Early_morning & IMPROPER LANE USAGE or PHYSICAL CONDITION OF DRIVER = 0.03, Morning & DISTRACTION - FROM INSIDE VEHICLE = 0.02, Night & FAILING TO REDUCE SPEED TO AVOID CRASH = 0.22
- CRASH_HOUR & INTERSECTION_RELATED_I: Night & Y = 0.36
- CRASH_HOUR & WEATHER_CONDITION: Night & SNOW = 0.05
- PRIM_CONTRIBUTORY_CAUSE & AGE_BIN: FAILING TO REDUCE SPEED TO AVOID CRASH & between 19 and 30 = 0.11
- PRIM_CONTRIBUTORY_CAUSE & SEX: FAILING TO REDUCE SPEED TO AVOID CRASH & F = 0.17
- PRIM_CONTRIBUTORY_CAUSE & CRASH_HOUR: FAILING TO REDUCE SPEED TO AVOID CRASH & Night = 0.22
- PRIM_CONTRIBUTORY_CAUSE & INTERSECTION_RELATED_I: FAILING TO REDUCE SPEED TO AVOID CRASH & Y = 0.20
- LIGHTING_CONDITION & PRIM_CONTRIBUTORY_CAUSE: DARKNESS, LIGHTED ROAD & PHYSICAL CONDITION OF DRIVER = 0.09, DAYLIGHT & FAILING TO REDUCE SPEED TO AVOID CRASH
- LIGHTING_CONDITION & INTERSECTION_RELATED_I: DARKNESS & Y = 0.05
- WEATHER_CONDITION	& AGE_BIN: RAIN & Over 60 = 0.05
- WEATHER_CONDITION & CRASH_HOUR: CLEAR & Night = 0.47